In [14]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [15]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'

In [16]:

def process_red(frame: np.ndarray, lower:np.ndarray = None, upper:np.ndarray = None) -> list:
    """Process single frame and return new frame with detected red objects."""
    frameHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(frameHSV, lower, upper)
    frameResult = cv2.bitwise_and(frame, frame, mask=mask)
    frameResult = cv2.morphologyEx(frameResult, cv2.MORPH_CLOSE, np.ones((7, 7), np.uint8), iterations=2)
    frameResult = cv2.dilate(frameResult, np.ones((5, 5), np.uint8))
    frameResult = cv2.dilate(frameResult, np.ones((5, 5), np.uint8))
    frameResult = cv2.medianBlur(frameResult, 7)


    frame = cv2.resize(frame, (0, 0), fx=1, fy=1)
    frameHSV = cv2.resize(frameHSV, (0, 0), fx=0.4, fy=0.4)
    mask = cv2.resize(mask, (0, 0), fx=0.4, fy=0.4)
    frameResult = cv2.resize(frameResult, (0, 0), fx=1, fy=1)


    result2 = frameResult.copy()
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(cv2.cvtColor(result2, cv2.COLOR_BGR2GRAY))
    objects_on_screen = num_labels > 1
    if objects_on_screen:
        result = []
        for object_index in range(1, num_labels):
            x, y, w, h, area = stats[object_index]
            if area > 20:
                result.append(stats[object_index])
        return result
    return []

In [17]:
def process_yellow(current_frame: np.ndarray, lower_range:np.ndarray = None, upper_range:np.ndarray = None) -> list:
    """Process single frame and return new frame with detected yellow objects."""
    
    hsv_image = cv2.cvtColor(current_frame, cv2.COLOR_BGR2HSV)
    
    if lower_range is None:
        lower_range = np.array([0, 90, 160])
        
    if upper_range is None:
        upper_range = np.array([30, 255, 255])

    binary_image = cv2.inRange(hsv_image, lower_range, upper_range)
    median = cv2.medianBlur(binary_image, 5)
    eroded = cv2.erode(median, np.ones((3, 3)))
    dilated = cv2.dilate(eroded, np.ones((5, 5)))
    dilated = cv2.erode(dilated, np.ones((5, 5)))

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(dilated)
    
    for i in range(1, num_labels):
        areas = stats[:, -1]  # Get the areas of all labels
        top_labels = np.argsort(areas)[::-1][:6]  # Get the indices of the top 5 labels with the largest areas
        result = []
        for i in top_labels:
            x, y, w, h, area = stats[i]
            if 8 < area < 150:
                result.append(stats[i])
        return result
    
    return []


In [18]:
def process_video(file: str, lower_y = None, upper_y = None, lower_r = None, upper_r = None) -> None:
    """Process video file and display it with detected objects."""
    capture = cv2.VideoCapture(file)

    while capture.isOpened():
        has_next_frame, current_frame = capture.read()
        if not has_next_frame:
            break

        yellow = process_yellow(current_frame, lower_y, upper_y)
        red = process_red(current_frame, lower_r, upper_r)
        
        for rect in yellow:
            x, y, w, h, area = rect
            cv2.rectangle(current_frame, (x - 5, y - 5), (x + w + 5, y + h + 5), (0, 255, 0), 2)
            cv2.putText(current_frame, f'{area}', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        for rect in red:
            x, y, w, h, area = rect
            cv2.rectangle(current_frame, (x - 5, y - 5), (x + w + 5, y + h + 5), (0, 0, 255), 2)
            cv2.putText(current_frame, f'{area}', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
        cv2.imshow("Labeled", current_frame)
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()

In [21]:
yellow_lower = np.array([0, 0, 230])
yellow_upper = np.array([35, 255, 255])

red_lower = np.array([139, 114, 151])
red_upper = np.array([179, 197, 212])

In [22]:
process_video('video_2.mp4', yellow_lower, yellow_upper, red_lower, red_upper)